<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 9</p>

In [4]:
# For this example, we will use K-Means Clustering Project database from Kaggle (https://www.kaggle.com/faressayah/k-means-clustering-private-vs-public-universities)
# We actually have the labels for this data set, but we will NOT use them for the KMeans clustering algorithm, since that is an unsupervised learning algorithm.
# As we will shortly see, we have a data frame with 777 observations on 18 variables.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
df = pd.read_csv('College_Data',index_col=0)
df.columns

Index(['apps', 'accept', 'enroll', 'top10perc', 'top25perc', 'f_undergrad',
       'p_undergrad', 'outstate', 'room_board', 'books', 'personal', 'phd',
       'terminal', 's_f_ratio', 'perc_alumni', 'expend', 'grad_rate'],
      dtype='object')

In [6]:
df.loc['Cazenovia College', 'Grad.Rate'] = 100  

# Try removing various columns (features) from the dataset and examin if it improves/degrades your K-Means model performance, or it may have little impact.
# Report 10 cases where you removed one or more features and indicate how it impacted the model performance.

# Standardize the features
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Function to perform clustering and evaluate
def evaluate_kmeans(data, features):
    X = data[features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    kmeans = KMeans(n_clusters=2, random_state=42)
    clusters = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, clusters)
    return score

# Base features
features = ['Apps', 'Accept', 'Enroll', 'Top10perc', 'Top25perc', 'F.Undergrad', 
           'P.Undergrad', 'Outstate', 'Room.Board', 'Books', 'Personal', 'PhD', 
           'Terminal', 'S.F.Ratio', 'perc.alumni', 'Expend', 'Grad.Rate']

# Test different feature combinations
print("Baseline (all features):", evaluate_kmeans(df, features))

# Case 1: Remove enrollment related features
features1 = [f for f in features if f not in ['Enroll', 'F.Undergrad', 'P.Undergrad']]
print("\nCase 1 - Without enrollment features:", evaluate_kmeans(df, features1))

# Case 2: Only academic features
features2 = ['Top10perc', 'Top25perc', 'PhD', 'Terminal', 'S.F.Ratio', 'Grad.Rate']
print("\nCase 2 - Only academic features:", evaluate_kmeans(df, features2))

# Case 3: Only financial features
features3 = ['Outstate', 'Room.Board', 'Books', 'Personal', 'Expend']
print("\nCase 3 - Only financial features:", evaluate_kmeans(df, features3))

# Case 4: Without cost features
features4 = [f for f in features if f not in ['Outstate', 'Room.Board', 'Books', 'Personal']]
print("\nCase 4 - Without cost features:", evaluate_kmeans(df, features4))

# Case 5: Only admission features
features5 = ['Apps', 'Accept', 'Top10perc', 'Top25perc']
print("\nCase 5 - Only admission features:", evaluate_kmeans(df, features5))

# Case 6: Without faculty features
features6 = [f for f in features if f not in ['PhD', 'Terminal', 'S.F.Ratio']]
print("\nCase 6 - Without faculty features:", evaluate_kmeans(df, features6))

# Case 7: Only performance indicators
features7 = ['Top10perc', 'Top25perc', 'Grad.Rate', 'perc.alumni']
print("\nCase 7 - Only performance indicators:", evaluate_kmeans(df, features7))

# Case 8: Without application features
features8 = [f for f in features if f not in ['Apps', 'Accept']]
print("\nCase 8 - Without application features:", evaluate_kmeans(df, features8))

# Case 9: Core features only
features9 = ['Top10perc', 'Outstate', 'PhD', 'S.F.Ratio', 'Grad.Rate']
print("\nCase 9 - Core features only:", evaluate_kmeans(df, features9))

# Case 10: Minimal feature set
features10 = ['Top25perc', 'Outstate', 'Grad.Rate']
print("\nCase 10 - Minimal feature set:", evaluate_kmeans(df, features10))

KeyError: 'Grad.Rate'